# Notebook for testing various things

In [1]:
import os
from pyspark.sql import SparkSession
import pandas as pd
from cassandra.cluster import Cluster

# Set pyspark env
os.environ["PYSPARK_PYTHON"] = "python"

spark = SparkSession.builder.appName('SparkCassandraApp').\
    config('spark.jars.packages', 'com.datastax.spark:spark-cassandra-connector_2.12:3.4.1').\
    config('spark.cassandra.connection.host', 'localhost').\
    config('spark.sql.extensions', 'com.datastax.spark.connector.CassandraSparkExtensions').\
    config('spark.sql.catalog.mycatalog', 'com.datastax.spark.connector.datasource.CassandraCatalog').\
    config('spark.cassandra.connection.port', '9042').getOrCreate()


cluster = Cluster(['localhost'], port=9042)
session = cluster.connect()
session.set_keyspace('compulsory')

def _get_df(table_name):

    (spark.read.format("org.apache.spark.sql.cassandra")
    .options(table=table_name, keyspace="compulsory")
    .load()
    .createOrReplaceTempView(table_name))

    df = spark.sql(f"select * from {table_name}").toPandas()
    return df


In [17]:
# lets start by checking whats in the database
table_name = 'fish_data_full'
fish_data = _get_df(table_name)
fish_data

,localityweekid,avgadultfemalelice,hascleanerfishdeployed,hasila,hasmechanicalremoval,haspd,hasreportedlice,hassalmonoids,hassubstancetreatments,infilteredselection,...,isonland,isslaughterholdingcage,lat,localityno,lon,municipality,municipalityno,name,week,year
0,1339821,NaN,False,False,False,False,False,False,False,True,...,False,False,58.251167,45015,6.792650,Kvinesdal,4227,Sagebukta,51,2021
1,205786,NaN,False,False,False,False,False,True,False,True,...,False,False,65.661636,24915,12.083917,Vega,1815,Bøbukta,50,2015
2,797343,NaN,False,False,False,False,False,False,False,True,...,True,False,59.278751,14297,5.508750,Tysvær,1146,"Iglamyr, Kårstø",8,2014
3,1474271,0.0,False,False,False,False,True,True,False,True,...,False,False,58.024300,30657,7.026800,Lyngdal,4225,Revøy,44,2022
4,316224,NaN,False,False,False,False,False,True,False,True,...,False,True,62.845501,27395,6.778383,Aukra,1547,Rindarøy,16,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635723,211891,NaN,False,False,False,False,False,True,False,True,...,False,False,62.615616,12841,7.276500,Rauma,1539,Kormeset,20,2013
635724,870508,NaN,False,False,False,False,False,False,False,True,...,True,False,60.517090,11748,4.903853,Øygarden,1259,Svartevika,35,2014
635725,267888,NaN,False,False,False,False,False,True,False,True,...,False,False,70.063782,10576,21.903450,Kvænangen,1943,Hjelnes,44,2016
635726,1464128,NaN,False,False,False,False,False,True,False,True,...,False,False,69.187119,10544,17.015083,Senja,5421,Flakstadvåg,41,2022


In [18]:
fish_data.isnull().all().to_list()

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

In [23]:
table_name = 'lice_data_full'

lice_data = _get_df(table_name)
lice_data

,id,avgadultfemalelice,avgmobilelice,avgstationarylice,hasbathtreatment,hascleanerfishdeployed,hasinfeedtreatment,hasmechanicalremoval,hasreportedlice,hassalmonoids,isfallow,isslaughterholdingcage,localityno,seatemperature,week,year
0,75236,1.93,12.43,3.13,False,False,False,False,True,True,False,False,23695,8.2,5,2015
1,75232,0.37,1.25,2.72,False,False,False,False,True,True,False,False,23695,7.0,1,2015
2,75282,0.00,0.91,1.27,False,False,False,False,True,True,False,False,23695,8.8,51,2015
3,75261,NaN,NaN,NaN,False,False,False,False,False,True,True,False,23695,NaN,30,2015
4,75240,0.19,2.28,1.18,False,False,False,False,True,True,False,False,23695,7.0,9,2015
5,75262,NaN,NaN,NaN,False,False,False,False,False,True,True,False,23695,NaN,31,2015
6,75277,0.01,0.04,0.02,False,False,False,False,True,True,False,False,23695,11.0,46,2015
7,75273,0.00,0.13,0.12,False,False,False,False,True,True,False,False,23695,13.0,42,2015
8,75242,0.47,4.07,1.90,True,False,False,False,True,True,False,False,23695,6.3,11,2015
9,75266,NaN,NaN,NaN,False,False,False,False,False,True,True,False,23695,NaN,35,2015


In [4]:
# from data_utils import *

# get_one_year_lice_data(10335, 2022, access_token= get_access_token())

WritingToDatabaseError: Error writing to database

In [10]:
def get_one_year_lice_data(locality, year, access_token):
    """
    Function to get all lice count data from Barentswatch API limited to one year.

    Parameters:
    -----------
    localty : int
        Localty number
    year : int
        Year of data
    access_token : str
        Access token from Barentswatch API
    Returns:
    --------
    df: pandas dataframe with data
    """

    #if year < 2010 or year > 2023:
     #   raise InvalidYearError("Year invalid")


    # Set list of weeks (1-52).
    weeks = np.arange(1, 53)
    df = pd.DataFrame()

    for week in weeks:
        data = get_one_week_lice_data(localty = locality, year = year, week = week, access_token = access_token)["localityWeek"]
        for key, value in data.items():
            # Set to list to make it compatible to convert to pandas dataframe
            data[key] = [value]
        # Dropping columns that contain purely None and nested dictionaries
        data = pd.DataFrame(data).drop(columns = ["bathTreatments", "cleanerFish", "inFeedTreatments", \
                                                "mechanicalRemoval", "timeSinceLastChitinSynthesisInhibitorTreatment"]) 
        data["year"] = year
        data["week"] = week
        df = pd.concat([df, data], ignore_index=True)
    # Lowercase column names
    df.columns = df.columns.str.lower()
    return df

df = get_one_year_lice_data(10423, 2015, access_token= get_access_token())
df

,id,localityno,year,week,hasreportedlice,hasmechanicalremoval,hasbathtreatment,hasinfeedtreatment,hascleanerfishdeployed,isfallow,avgadultfemalelice,avgmobilelice,avgstationarylice,seatemperature,hassalmonoids,isslaughterholdingcage
0,325732,10423,2015,1,False,False,False,False,False,True,None,None,None,None,True,False
1,325733,10423,2015,2,False,False,False,False,False,True,None,None,None,None,True,False
2,325734,10423,2015,3,False,False,False,False,False,True,None,None,None,None,True,False
3,325735,10423,2015,4,False,False,False,False,False,True,None,None,None,None,True,False
4,325736,10423,2015,5,False,False,False,False,False,True,None,None,None,None,True,False
5,325737,10423,2015,6,False,False,False,False,False,True,None,None,None,None,True,False
6,325738,10423,2015,7,False,False,False,False,False,True,None,None,None,None,True,False
7,325739,10423,2015,8,False,False,False,False,False,True,None,None,None,None,True,False
8,325740,10423,2015,9,False,False,False,False,False,True,None,None,None,None,True,False
9,325741,10423,2015,10,False,False,False,False,False,True,None,None,None,None,True,False


In [14]:
from exceptions import *
if True in df.isnull().all().to_list():
    raise ValueError("No data for this year")

ValueError: No data for this year

In [29]:
# select locality fish_data 45083

fish_data.loc[fish_data['localityno'] == 45083]

,localityweekid,avgadultfemalelice,hascleanerfishdeployed,hasila,hasmechanicalremoval,haspd,hasreportedlice,hassalmonoids,hassubstancetreatments,infilteredselection,...,isonland,isslaughterholdingcage,lat,localityno,lon,municipality,municipalityno,name,week,year
0,1439901,NaN,False,False,False,False,False,True,False,True,...,False,False,69.987152,45083,20.664618,Skjervøy,5427,Vorterøyskagen,31,2022
26491,1450117,NaN,False,False,False,False,False,True,False,True,...,False,False,69.987152,45083,20.664618,Skjervøy,5427,Vorterøyskagen,35,2022
27777,1286940,NaN,False,False,False,False,False,True,False,True,...,False,False,69.987152,45083,20.664618,Skjervøy,5427,Vorterøyskagen,33,2021
35305,1417622,NaN,False,False,False,False,False,True,False,True,...,False,False,69.987152,45083,20.664618,Skjervøy,5427,Vorterøyskagen,24,2022
39153,1351527,NaN,False,False,False,False,False,True,False,True,...,False,False,69.987152,45083,20.664618,Skjervøy,5427,Vorterøyskagen,3,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590560,1466566,NaN,False,False,False,False,False,True,False,True,...,False,False,69.987152,45083,20.664618,Skjervøy,5427,Vorterøyskagen,41,2022
591661,1434792,NaN,False,False,False,False,False,True,False,True,...,False,False,69.987152,45083,20.664618,Skjervøy,5427,Vorterøyskagen,29,2022
598277,1471678,NaN,False,False,False,False,False,True,False,True,...,False,False,69.987152,45083,20.664618,Skjervøy,5427,Vorterøyskagen,43,2022
601369,1336826,NaN,False,False,False,False,False,True,False,True,...,False,False,69.987152,45083,20.664618,Skjervøy,5427,Vorterøyskagen,50,2021


In [31]:
# select the unique rows based on localityno 
fish_data_unique = fish_data.drop_duplicates(subset=['localityno']).reset_index(drop=True)
fish_data_unique

,localityweekid,avgadultfemalelice,hascleanerfishdeployed,hasila,hasmechanicalremoval,haspd,hasreportedlice,hassalmonoids,hassubstancetreatments,infilteredselection,...,isonland,isslaughterholdingcage,lat,localityno,lon,municipality,municipalityno,name,week,year
0,1439901,NaN,False,False,False,False,False,True,False,True,...,False,False,69.987152,45083,20.664618,Skjervøy,5427,Vorterøyskagen,31,2022
1,722007,NaN,False,False,False,False,False,False,False,True,...,True,False,61.917191,13619,5.297075,Vågsøy,1439,Rødhjelle I,20,2015
2,99938,NaN,False,False,False,False,False,True,False,True,...,False,False,64.958992,10417,11.611913,Nærøy,1751,Storbukta,35,2013
3,1243220,0.0,False,False,False,False,True,True,False,True,...,False,False,61.684849,24615,4.915633,Kinn,4602,Grunnsøya,19,2021
4,851694,NaN,False,False,False,False,False,False,False,True,...,False,False,59.073769,24436,5.432783,Kvitsøy,1144,Nordre Hestholmen S,19,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2558,822114,NaN,False,False,False,False,False,False,False,True,...,False,False,59.844532,11505,5.251183,Bømlo,1219,Austnesvågen,1,2012
2559,390746,NaN,False,False,False,False,False,True,False,True,...,False,False,63.281235,30056,8.332067,Aure,1576,Steinholmen,1,2012
2560,368136,NaN,False,False,False,False,False,True,False,True,...,False,False,63.134968,30016,7.985067,Aure,1576,Kråkøya,1,2012
2561,407238,NaN,False,False,False,False,False,True,False,True,...,False,True,69.611771,23315,19.874166,Lyngen,1938,Storsteinnes,1,2012


In [23]:
# take the columns and convert them to a dict with key as index (0,1,..p) and value as column name. # sort it by alphabetical order.
cols = ['lat', 'lon', 'localityweekid', 'localityno', 'municipality', 'municipalityno', 'name', 'week', 'year']

test = fish_data.drop(columns=cols)
columns = test.columns.sort_values().tolist()
columns = dict(enumerate(columns))
columns

{0: 'avgadultfemalelice',
 1: 'hascleanerfishdeployed',
 2: 'hasila',
 3: 'hasmechanicalremoval',
 4: 'haspd',
 5: 'hasreportedlice',
 6: 'hassalmonoids',
 7: 'hassubstancetreatments',
 8: 'infilteredselection',
 9: 'isfallow',
 10: 'isonland',
 11: 'isslaughterholdingcage'}

In [24]:
print(columns)

{0: 'avgadultfemalelice', 1: 'hascleanerfishdeployed', 2: 'hasila', 3: 'hasmechanicalremoval', 4: 'haspd', 5: 'hasreportedlice', 6: 'hassalmonoids', 7: 'hassubstancetreatments', 8: 'infilteredselection', 9: 'isfallow', 10: 'isonland', 11: 'isslaughterholdingcage'}


In [6]:
import geopandas
test = fish_data.copy()

test_fish = geopandas.GeoDataFrame(
    test, geometry=geopandas.points_from_xy(test.lat, test.lon), crs="EPSG:4326"
)

test_fish["lat"] = (
    test_fish.to_crs(test_fish.estimate_utm_crs()).simplify(1000).to_crs(test_fish.crs)
)

: 

In [1]:
test_fish

NameError: name 'test_fish' is not defined

In [5]:
fish_data.columns

Index(['localityweekid', 'avgadultfemalelice', 'hascleanerfishdeployed',
       'hasila', 'hasmechanicalremoval', 'haspd', 'hasreportedlice',
       'hassalmonoids', 'hassubstancetreatments', 'infilteredselection',
       'isfallow', 'isonland', 'isslaughterholdingcage', 'lat', 'localityno',
       'lon', 'municipality', 'municipalityno', 'name', 'week', 'year'],
      dtype='object')

In [14]:
import numpy as np
fish_data['lat'] = fish_data['lat'].astype(np.float64)

In [16]:
fish_data['year'].value_counts()

2012    99549
2013    92282
2014    90528
2022    89084
2015    88664
Name: year, dtype: int64

In [14]:
class InvalidYearError(Exception):
    def __init__(self, message="Invalid year"):
        self.message = message
        super().__init__(self.message)

class DataExistsError(Exception):
    def __init__(self, message="Data already exists in database"):
        self.message = message
        super().__init__(self.message)

class WritingToDatabaseError(Exception):
    def __init__(self, message="Error writing to database"):
        self.message = message
        super().__init__(self.message)

def test(year):
    if year < 2010 or year > 2023:
        raise InvalidYearError
test(209)

InvalidYearError: Invalid year

In [36]:
years = [2014, 2004, 2015]
# sort the years 
years.sort()

test = {str(i): int(years[i]) for i in range(len(years))}

test

{'0': 2004, '1': 2014, '2': 2015}

In [32]:
# sort test by value
sorted_test = dict(sorted(test.items(), key=lambda x: x[1]))
sorted_test

{'1': 2004, '0': 2014, '2': 2015}

In [33]:
# next, replace the keys with values ranging from 0 to len(test)
sorted_test = {i: sorted_test[str(i)] for i in range(len(sorted_test))}
sorted_test

{0: 2014, 1: 2004, 2: 2015}

In [35]:
years_dict = {str(i): int(years[i]) for i in range(len(years))}
years_dict_sorted_vals = dict(sorted(years_dict.items(), key=lambda item: item[1]))
years_dict_sorted_keys = {i: years_dict_sorted_vals[str(i)] for i in range(len(years_dict_sorted_vals))}
years_dict_sorted_keys

{0: 2014, 1: 2004, 2: 2015}

In [3]:
import streamsync as ss
import plotly.express as px

def _update_plotly_fish(state, data):
    fish_data = data
    selected_num = state["plotly_settings"]["selected_num"]
    sizes = [10]*len(fish_data)

    if selected_num != -1:
        sizes[selected_num] = 20

    fig_fish = px.scatter_mapbox(
        fish_data,
        lat="lat",
        lon="lon",
        hover_name="name",
        hover_data=["municipality","lat","lon"],
        color_discrete_sequence=["darkgreen"],
        zoom=14,
        height=600,
        width=700,
    )
    overlay = fig_fish['data'][0]
    overlay['marker']['size'] = sizes
    fig_fish.update_layout(mapbox_style="open-street-map")
    fig_fish.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    state["plotly_settings"]["fish_map"] = fig_fish

initial_state = ss.init_state({
"plotly_settings": {"selected_name": "Click to select",
                      "selected_num": -1,
                      "fish_map": None
    }
})


_update_plotly_fish(initial_state, fish_data)

In [7]:
# select the first 10 rows
fish_data_10 = fish_data.iloc[:1000]

In [22]:
import plotly.express as px

fig_fish = px.scatter_mapbox(
fish_data_10,
lat="lat",
lon="lon",
hover_name="name",
hover_data=["municipality","lat","lon"],
color_discrete_sequence=["darkgreen"],
zoom=14,
height=600,
width=700,
)
overlay = fig_fish['data'][0]
overlay['marker']['size'] = 5
fig_fish.update_layout(mapbox_style="open-street-map")
fig_fish.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

In [23]:
# list fig_fish contents

current_viewport = fig_fish.layout.mapbox

In [24]:
import plotly.express as px

new_fig_fish = px.scatter_mapbox(
fish_data_10,
lat="lat",
lon="lon",
hover_name="name",
hover_data=["municipality","lat","lon"],
color_discrete_sequence=["darkgreen"],
zoom=14,
height=600,
width=700,
)
overlay = fig_fish['data'][0]
overlay['marker']['size'] = 5
new_fig_fish.update_layout(mapbox_style="open-street-map")
new_fig_fish.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
new_fig_fish.layout.mapbox = current_viewport

In [25]:
fig_fish.show()